#### Pipeline for identifying same cell clusters
This notebook shows some of the work for identifying and labeling same cell clusters. It's not a complete representation of the pipeline, apologies. But I'll do my best to describe the protocol here and the key steps.

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib widget

from time import perf_counter
import numpy as np
from matplotlib import pyplot as plt
from vrAnalysis import database
from vrAnalysis2.sessions import create_b2session
from vrAnalysis2.helpers.debug import Timer
from vrAnalysis2.analysis.same_cell_candidates import SameCellProcessor, get_connected_groups
from vrAnalysis2.analysis.same_cell_candidates import (
    plot_correlation_vs_distance,
    plot_plane_pair_histograms,
    plot_cluster_size_distribution,
    plot_distance_distribution,
    plot_roi_removal_analysis,
    ClusterExplorer,
)

sessiondb = database.vrDatabase('vrSessions')

In [2]:
random_choice = False
if random_choice:
    import random
    session = random.choice(sessiondb.iterSessions(imaging=True))
    mouse_name = session.mouseName
    date = session.dateString
    session_id = session.sessionid
else:
    mouse_name = "ATL012"
    date = "2023-01-20"
    session_id = "702"

spks_type = "corrected"
session = create_b2session(mouse_name, date, session_id, dict(spks_type=spks_type))
print(session)

B2Session(mouse_name='ATL012', date='2023-01-20', session_id='702')


In [10]:
scp = SameCellProcessor(session).load_data()

In [11]:
explorer = ClusterExplorer(scp)

In [12]:
explorer.deploy(env="notebook")

In [18]:
show_cluster_size = False
if show_cluster_size:
    plot_cluster_size_distribution(scp, verbose=False)

In [9]:
show_correlation_vs_distance = True
if show_correlation_vs_distance:
    viewer = plot_correlation_vs_distance(scp)
    viewer.deploy(env="notebook")

No such comm: 908afe7039d743d99fb8666d5f2972de
No such comm: 908afe7039d743d99fb8666d5f2972de
No such comm: 908afe7039d743d99fb8666d5f2972de
No such comm: 908afe7039d743d99fb8666d5f2972de
No such comm: 908afe7039d743d99fb8666d5f2972de
No such comm: 908afe7039d743d99fb8666d5f2972de
No such comm: 908afe7039d743d99fb8666d5f2972de
No such comm: 908afe7039d743d99fb8666d5f2972de
No such comm: 908afe7039d743d99fb8666d5f2972de
No such comm: 908afe7039d743d99fb8666d5f2972de
